In [31]:
import numpy as np
from PIL import Image

In [32]:
res = 600
center = (-1.85406,0.00343)
zoom=2**5

In [33]:
coords = np.indices((res,res),dtype=np.int32).transpose(1,2,0)-(res/2)
coords [:,:,1] = (coords[:,:,1])
coords[:,:,0] = coords[:,:,0]

In [34]:
cpx = coords.copy().astype(complex)
cpx[:,:,1] = coords [:,:,1] * 1j
cpx = cpx/(res*zoom)
cpx[:,:,0] += (center[0])
cpx[:,:,1] += (center[1]) * 1j

In [35]:
z = np.concatenate([np.expand_dims(cpx.sum(axis=2),axis=2),coords,np.zeros((res,res,2))], axis=-1)

In [36]:
iterations = 600

In [37]:
z[:,:,3] = z[:,:,3]**2+z[:,:,0]
for i in range(0,iterations):
    z[:,:,3] = z[:,:,3]**2+z[:,:,0] * (z[:,:,3]!=0).astype(int)
    excaped = np.where(np.absolute(z[:,:,3]) > 2)
    #z[:,:,4] must show the i like i did earlier
    z[excaped[0],excaped[1],3] = 0
    z[excaped[0],excaped[1],4] = i

In [38]:
final = np.zeros((res,res,3))
values = z[..., 4].ravel()
xs = z[..., 1].astype(int).ravel() + (res/2)
ys = z[..., 2].astype(int).ravel() + (res/2)

/tmp/ipykernel_5791/1352648235.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  xs = z[..., 1].astype(int).ravel() + (res/2)
/tmp/ipykernel_5791/1352648235.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  ys = z[..., 2].astype(int).ravel() + (res/2)


In [39]:
j = np.column_stack((np.sin(values*0.1+0.1)*127 + 128,np.cos(values*0.1+0.1)*127 + 128,np.full(values.shape,255)))
j[values==0] = [0,0,0]

In [40]:
final[xs.astype(int),ys.astype(int)] = j

/tmp/ipykernel_5791/700153843.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  final[xs.astype(int),ys.astype(int)] = j


In [41]:
Image.fromarray(final.astype(np.uint8)).save("mandelbrot.png")